# Diamonds model

## Laden der Daten

Zum Beginn lade ich meinen Diamanten-Datensatz

In [86]:
import pandas as pd
import os

diamonds = pd.read_csv("diamonds.csv")
diamonds.head(100)

,carat,cut,color,clarity,depth,table,x,y,z,price
0,0.23,Ideal,E,SI2,61.5,55.0,3.95,3.98,2.43,326
1,0.21,Premium,E,SI1,59.8,61.0,3.89,3.84,2.31,326
2,0.23,Good,E,VS1,56.9,65.0,4.05,4.07,2.31,327
3,0.29,Premium,I,VS2,62.4,58.0,4.20,4.23,2.63,334
4,0.31,Good,J,SI2,63.3,58.0,4.34,4.35,2.75,335
...,...,...,...,...,...,...,...,...,...,...
95,0.70,Good,E,VS2,57.5,58.0,5.85,5.90,3.38,2759
96,0.70,Good,F,VS1,59.4,62.0,5.71,5.76,3.40,2759
97,0.96,Fair,F,SI2,66.3,62.0,6.27,5.95,4.07,2759
98,0.73,Very Good,E,SI1,61.6,59.0,5.77,5.78,3.56,2760


## *features* und *targets*

Dann definiere ich meine features(X) und meine targets (y).

In [87]:
diamonds["size"] = diamonds["x"] * diamonds["y"] * diamonds["z"]
X = diamonds[["carat", "cut", "color", "clarity", "depth", "table", "size"]]
y = diamonds[["price"]]

X.head(100)

,carat,cut,color,clarity,depth,table,size
0,0.23,Ideal,E,SI2,61.5,55.0,38.202030
1,0.21,Premium,E,SI1,59.8,61.0,34.505856
2,0.23,Good,E,VS1,56.9,65.0,38.076885
3,0.29,Premium,I,VS2,62.4,58.0,46.724580
4,0.31,Good,J,SI2,63.3,58.0,51.917250
...,...,...,...,...,...,...,...
95,0.70,Good,E,VS2,57.5,58.0,116.660700
96,0.70,Good,F,VS1,59.4,62.0,111.824640
97,0.96,Fair,F,SI2,66.3,62.0,151.837455
98,0.73,Very Good,E,SI1,61.6,59.0,118.728136


## Preprocessing

Hier prüfe ich, ob es unvollständige Datensätze gibt. Es gibt keine.

In [88]:
len(X[X.isna().any(axis=1)])

0

Danach führe ich für die Spalten "cut", "color" und "clarity" ein OneHotEncoding durch, damit die verschiedenen Kategorien als Zahlen einfacher verarbeitet werden können.

In [89]:
from sklearn.preprocessing import OrdinalEncoder

X.loc[:, ["cut"]] = OrdinalEncoder().fit_transform(X[["cut"]])
X.loc[:, ["color"]] = OrdinalEncoder().fit_transform(X[["color"]])
X.loc[:, ["clarity"]] = OrdinalEncoder().fit_transform(X[["clarity"]])

X.head(100)

,carat,cut,color,clarity,depth,table,size
0,0.23,2.0,1.0,3.0,61.5,55.0,38.202030
1,0.21,3.0,1.0,2.0,59.8,61.0,34.505856
2,0.23,1.0,1.0,4.0,56.9,65.0,38.076885
3,0.29,3.0,5.0,5.0,62.4,58.0,46.724580
4,0.31,1.0,6.0,3.0,63.3,58.0,51.917250
...,...,...,...,...,...,...,...
95,0.70,1.0,1.0,5.0,57.5,58.0,116.660700
96,0.70,1.0,2.0,4.0,59.4,62.0,111.824640
97,0.96,0.0,2.0,3.0,66.3,62.0,151.837455
98,0.73,4.0,1.0,2.0,61.6,59.0,118.728136


Hier skaliere ich die Spalten "carat", "depth", "table", "size", damit beim Vergleichen der verschiedenen Zeilen alle Spalten gleich ins Gewicht fallen.

In [96]:
columns_to_scale = ["carat", "depth", "table", "size"]

for column in columns_to_scale:
    column_min = X[column].min()
    column_max = X[column].max()
    X.loc[:,[column]] = (X[column] - column_min) / (column_max - column_min)

X.head(100)

,carat,cut,color,clarity,depth,table,size
0,0.006237,2.0,1.0,3.0,0.513889,0.230769,0.009947
1,0.002079,3.0,1.0,2.0,0.466667,0.346154,0.008985
2,0.006237,1.0,1.0,4.0,0.386111,0.423077,0.009914
3,0.018711,3.0,5.0,5.0,0.538889,0.288462,0.012166
4,0.022869,1.0,6.0,3.0,0.563889,0.288462,0.013518
...,...,...,...,...,...,...,...
95,0.103950,1.0,1.0,5.0,0.402778,0.288462,0.030376
96,0.103950,1.0,2.0,4.0,0.455556,0.365385,0.029116
97,0.158004,0.0,2.0,3.0,0.647222,0.365385,0.039535
98,0.110187,4.0,1.0,2.0,0.516667,0.307692,0.030914


## *test-train-split*

Nun teile ich meinen Diamanten Datensatz in *test* und *train* auf mit einem Verhältnis 20:80.

In [97]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size= 0.2, random_state = 36
)

len(X_train), len(X_test)

(43152, 10788)

## Algorithmuswahl

Bei der Wahl des Algorithmus teste ich verschiedene Regressionsalgorithmen, weil wir den Preis, also eine Zahl, berechnen wollen.

In [98]:
from sklearn.neural_network import MLPRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
 
algorithms = {
    "Nearest Neighbors": KNeighborsRegressor(4),
    "Stochastic Gradient Descent": SGDRegressor(max_iter=2000, tol=1e-3),
    "Decision Tree": DecisionTreeRegressor(max_depth=5),
    "Random Forest": RandomForestRegressor(max_depth=5, n_estimators=10, max_features=1)
}

for name, algorithm in algorithms.items():
    algorithm.fit(X_train, y_train.values.ravel())
    score = algorithm.score(X_test, y_test)
    print(name, round(score,2))

Nearest Neighbors 0.96
Stochastic Gradient Descent 0.87
Decision Tree 0.92
Random Forest 0.81


Da der *Nearest Neighbors* Algoritmus die höchste Punktzahl hat, wähle ich diesen.

## Modell speichern

In diesem Schritt speichere ich meinen obigen Algorithmus für spätere Projekte.

In [99]:
import joblib

joblib.dump(algorithms["Nearest Neighbors"], 'diamonds_price_algorithm.joblib')
diamonds_price_algorithm = joblib.load('diamonds_price_algorithm.joblib')
diamonds_price_algorithm

KNeighborsRegressor(n_neighbors=4)

## Testen

Zum Abschluss überprüfe ich meinen trainierten Algorithmus manuell auf seine Sinnhaftigkeit.

In [100]:
tests = X.sample(3, random_state=36)
print(tests)

          carat  cut color clarity     depth     table      size
53329  0.106029  2.0   1.0     2.0  0.513889  0.269231  0.030039
52050  0.122661  1.0   5.0     5.0  0.430556  0.365385  0.033159
25759  0.442827  3.0   6.0     2.0  0.469444  0.307692  0.101892


In [101]:
predictions = diamonds_price_algorithm.predict(tests)
print(predictions)

[ 2517.5   3436.5  14090.25]


Der erste Datensatz hat den geringsten Karatwert. Der Algorithmus sagt einen Preis von **2517.5** voraus. Dies ist verständlich, da Diamanten mit einem geringen Karatwert weniger kosten.

Der zweite Datensatz hat einen ähnlichen Karatwert und eine ähnliche Grösse, wie der erste kostet aber trotzdem **3436.50**. Der Algorithmus erkennt korrekt, dass dieser zweite Diamant wertvoller als der erste ist, weil die Farbe und Klarheit besser ist.

Der dritte Datensatz ist am grössten. Der Algorithmus sagt einen Preis von **14090.25** voraus. Dies ist verständlich, da grosse Diamanten mehr kosten.

## Zusammenfassung

Das Modell erledigt seine Aufgabe sehr gut. Der Algorithmus hat korrekt erkannt, dass grössere Diamanten mit einem grossen Karatwert teurer sind als kleine Diamanten mit einem tiefen Karatwert. Zudem erkennt er, dass auch Unterschiede in der Farbe und in der Klarheit eine Rolle spielen.